# New Angles from Right Range: Optimizing Car Sensor Positioning with D-Wave Hybrid Quantum Computers

### BMW Quantum Computing Challenge -- Round 2

#### Valter Uotila, [Unified Database Management Systems](https://www2.helsinki.fi/en/researchgroups/unified-database-management-systems-udbms/people#section-63562)
#### Sardana Ivanova, [Discovery Research Group](https://www2.helsinki.fi/en/researchgroups/computational-creativity-and-data-mining/people#section-102417)

#### Department of Computer Science, University of Helsinki

## Introduction to the implementation

We have been learning, testing and developing some basic quantum algorithms using D-waves own API but we are not yet very experienced with details related to these quantum computers.

The structure of the code is simple: first we import the data, then we create the binary variables as described in the proposal. After that we construct the three objective functions. Finally, we send the total objective functions to D-wave's quantum computer which solves it. The final result is printed.

## Initializing parameters

### Importing D-wave packages

In order to run the code here, you need to be able to successfully access D-wave's quantum cloud computing resources. You can see more info at https://cloud.dwavesys.com/. The reason is that, unfortunately, Amazon Bracket does not yet support D-wave's hybrid solvers.

In [1]:
import dimod

from dwave.system import LeapHybridSampler

import json
import csv

from itertools import combinations
import pprint
import os
import math

import warnings
#warnings.filterwarnings("ignore")

import numpy as np
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from sympy import *

notebook_path = os.path.abspath("main_3D_connecting_Dwave_Leap.ipynb")

### Global parameters

The parameters `enviroment_x`, `enviroment_y`, `enviroment_z` describe the dimensions of the environment. The environment is the space where the car is located in. Anything outside the environment is supposed not to be accessible by sensors. We took the values for these parameters from the data set `criticallity_grid` which BMW provided.

In [2]:
car_sample_accuracy = 500

angle_accuracy = 60

variables = dict()

all_variables = list()

criticallity_grid_abs_path = os.path.join(os.path.dirname(notebook_path), "sensor_position_data/criticallity_grid_0_5.csv")

xs, ys, zs = list(), list(), list()

with open(criticallity_grid_abs_path, newline='') as csvfile:
    records = iter(csv.reader(csvfile, delimiter=','))
    next(records)
    for row in records:
        xs.append(int(float(row[0])*100))
        ys.append(int(float(row[1])*100))
        zs.append(int(float(row[2])*100))
        
print("Corner points of the environment: ", max(xs), min(xs), max(ys), min(ys), max(zs), min(zs))

environment_x = abs(max(xs)) + abs(min(xs))
environment_y = abs(max(ys)) + abs(min(ys))
environment_z = abs(max(zs)) + abs(min(zs))

print("Environment x-coordinate: ", environment_x)
print("Environment y-coordinate: ", environment_y)
print("Environment z-coordinate: ", environment_z)

Corner points of the environment:  11600 -7500 5100 -5100 650 -50
Environment x-coordinate:  19100
Environment y-coordinate:  10200
Environment z-coordinate:  700


### Importing sensors

We created various sensors to demonstrate the code. It is difficult to say if these sensors have realistic values because we didn't have sensor examples in BMW data.

In [3]:
abs_sensors_file_path = os.path.join(os.path.dirname(notebook_path), "3d_example_data/3D_sensors_set_1.json")

sensor_types = {0: 'lidar', 1: 'radar', 2: 'camera', 3: 'ultrasound'}

f = open(abs_sensors_file_path)

sensor_root = json.load(f)
sensors = sensor_root["sensors"]

def get_sensor_price(sensor_id):
    for sensor in sensors:
        if sensor['id'] == sensor_id:
            return sensor['price']

print(json.dumps(sensors, indent=4, sort_keys=True))

[
    {
        "id": 1,
        "price": 100,
        "type": 1,
        "view": {
            "angle": {
                "horizontal": 10,
                "vertical": 50
            },
            "range": 2000
        }
    },
    {
        "id": 2,
        "price": 200,
        "type": 2,
        "view": {
            "angle": {
                "horizontal": 10,
                "vertical": 50
            },
            "range": 5000
        }
    },
    {
        "id": 3,
        "price": 100,
        "type": 3,
        "view": {
            "angle": {
                "horizontal": 10,
                "vertical": 50
            },
            "range": 7000
        }
    },
    {
        "id": 4,
        "price": 10,
        "type": 1,
        "view": {
            "angle": {
                "horizontal": 10,
                "vertical": 50
            },
            "range": 10000
        }
    },
    {
        "id": 5,
        "price": 100,
        "type": 2,
        "view": {
    

### Initializing allowed sensor positions on car

Here we utilize the data BMW provided. Compared to the first-round two-dimensional version, the car model in this second-round version is based on the allowed sensor positions data. In order to create variables, we need to sample some points from the surfaces. To sample points, we need some easier format for the surfaces than vector representation. Thus we calculate plane equations for the surfaces. The equation allows us to sample points when we are creating the variables.

In [4]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)


def test_vectors(v1, v2, a, b, c, cp):
    
    if not np.allclose([1,1,0,0], [v1.dot(v1), v2.dot(v2), v1.dot(v2), v1.dot(v2)]):
        print("Error in dot products!")
        print(v1.dot(v2))
        print(v1.dot(v2))
    
    if not np.allclose([1,0,0], [cp.dot(cp), cp.dot(v1), cp.dot(v2)]):
        print("Error in normal vector dot products!")
        print(cp.dot(v1))
        print(cp.dot(v2))
        
        
def plot_region(a, b, c, d):
    x = np.linspace(-1,1,10)
    y = np.linspace(-1,1,10)

    X,Y = np.meshgrid(x,y)
    if c != 0:
        Z = (d - a*X - b*Y) / c

        fig = plt.figure()
        ax = fig.gca(projection='3d')

        surf = ax.plot_surface(X, Y, Z)
        
        
def scatter_plot(points, save = False):
    fig = pyplot.figure()
    ax = Axes3D(fig)
    
    x_vals = [c[0] for c in points]
    y_vals = [c[1] for c in points]
    z_vals = [c[2] for c in points]
    
    #ax.scatter3D(x_vals, y_vals, z_vals, c=z_vals);
    
    ax.scatter(x_vals, y_vals, z_vals)
    #ax.set_xlabel('X Label')
    #ax.set_ylabel('Y Label')
    #ax.set_zlabel('Z Label')
    pyplot.show()
    

def plane_equation(c1, c2, c3):
    p1 = np.array([c1[0], c1[1], c1[2]])
    p2 = np.array([c2[0], c2[1], c2[2]])
    p3 = np.array([c3[0], c3[1], c3[2]])
    
    # These two vectors are in the plane
    v1 = unit_vector(p3 - p1)
    v2 = unit_vector(p2 - p1)
    
    # We modify the second vector so that it is orthogonal with the vector v1.
    # This is important so that we get 90 degree angle between the vectors and we
    # can use trigonometric functions.
    v2 -= v1.dot(v2)*v1
    
    # Cross product is a vector normal to the plane 
    # spanned by vectors v1 and v2
    cp = unit_vector(np.cross(v1, v2))
    cp = np.array([round(x, 7) for x in cp])
    a, b, c = cp
    
    # Testing vectors
    # test_vectors(v1, v2, a, b, c, cp)
    
    # This evaluates a * x3 + b * y3 + c * z3 which equals d
    d = np.dot(cp, p3)
    
    # print('Equation is {0}x + {1}y + {2}z = {3}'.format(a, b, c, d))
    # plot_region(a, b, c, d)
    
    return { 'span1': v1, 'span2': v2, 'normal_vector': cp, 'x': a, 'y': b, 'z': c, 'd': d }    

In the following, we calculate the planes for the different surfaces of the car. Besides, we characterize each plane with its plane equation (or line).

In [5]:
abs_position_file_path = os.path.join(os.path.dirname(notebook_path), "sensor_position_data/allowed_sensor_positions.csv")

allowed_sensor_positions = dict()

with open(abs_position_file_path, newline='') as csvfile:
    records = iter(csv.DictReader(csvfile, delimiter=';'))
    next(records)
    for row in records:
        c1 = (int(row["x1"]), int(row["y1"]), int(row["z1"]))
        c2 = (int(row["x2"]), int(row["y2"]), int(row["z2"]))
        c3 = (int(row["x3"]), int(row["y3"]), int(row["z3"]))
        c4 = (int(row["x4"]), int(row["y4"]), int(row["z4"]))
        
        region = row["Region"].lower()
        
        allowed_sensor_positions[region] = dict()
        allowed_sensor_positions[region]["corners"] = [c1, c2, c3, c4]
        allowed_sensors = [x.lower() for x in row["Allowed Sensors"].split(", ")]
        allowed_sensor_positions[region]["allowed_sensors"] = allowed_sensors
        
        equation = plane_equation(c1, c2, c3)
        allowed_sensor_positions[region]["equation"] = equation
        
        intervals = dict()
            
        if equation["x"] == 0:
            if c1[0] < c2[0]:
                intervals["x"] = range(c1[0], c2[0], car_sample_accuracy)
            elif c1[0] > c2[0]:
                intervals["x"] = range(c2[0], c1[0], car_sample_accuracy)
            elif c1[0] < c3[0]:
                intervals["x"] = range(c1[0], c3[0], car_sample_accuracy)
            elif c1[0] > c3[0]:
                intervals["x"] = range(c3[0], c1[0], car_sample_accuracy)
                
        if equation["y"] == 0:
            if c1[1] < c2[1]:
                intervals["y"] = range(c1[1], c2[1], car_sample_accuracy)
            elif c1[1] > c2[1]:
                intervals["y"] = range(c2[1], c1[1], car_sample_accuracy)
            elif c1[1] < c3[1]:
                intervals["y"] = range(c1[1], c3[1], car_sample_accuracy)
            elif c1[1] > c3[1]:
                intervals["y"] = range(c3[1], c1[1], car_sample_accuracy)
                
        if equation["z"] == 0:
            if c1[2] < c2[2]:
                intervals["z"] = range(c1[2], c2[2], car_sample_accuracy)
            elif c1[2] > c2[2]:
                intervals["z"] = range(c2[2], c1[2], car_sample_accuracy)
            elif c1[2] < c3[2]:
                intervals["z"] = range(c1[2], c3[2], car_sample_accuracy)
            elif c1[2] > c3[2]:
                intervals["z"] = range(c3[2], c1[2], car_sample_accuracy)
                
        allowed_sensor_positions[region]["fixed_intervals"] = intervals

#for_printing = allowed_sensor_positions[elem]
#pp = pprint.PrettyPrinter(width=41, compact=True)
for elem in allowed_sensor_positions:
    print(elem, allowed_sensor_positions[elem])
    print()
        

bonnet {'corners': [(-1400, 1000, 1200), (-1400, -1000, 1200), (0, -1000, 1000), (0, 1000, 1000)], 'allowed_sensors': ['lidar', 'camera'], 'equation': {'span1': array([ 0.57154761, -0.81649658, -0.08164966]), 'span2': array([-0.46666667, -0.33333333,  0.06666667]), 'normal_vector': array([-0.1414214,  0.       , -0.9899495]), 'x': -0.1414214, 'y': 0.0, 'z': -0.9899495, 'd': -989.9495000000001}, 'fixed_intervals': {'y': range(-1000, 1000, 500)}}

side mirror left {'corners': [(-1400, 1150, 1320), (-1400, 950, 1320), (-1400, 950, 1200), (-1400, 1150, 1200)], 'allowed_sensors': ['lidar', 'camera'], 'equation': {'span1': array([ 0.        , -0.85749293, -0.51449576]), 'span2': array([ 0.        , -0.26470588,  0.44117647]), 'normal_vector': array([-1., -0.,  0.]), 'x': -1.0, 'y': -0.0, 'z': 0.0, 'd': 1400.0}, 'fixed_intervals': {'y': range(950, 1150, 500), 'z': range(1200, 1320, 500)}}

side mirror right {'corners': [(-1400, -950, 1320), (-1400, -1150, 1320), (-1400, -1150, 1200), (-1400, 

### Initializing variables

Compared to the first-round, variables are updated to be triples `(x, y, i)` where `x` and `y` are points in the 3D-space and `i` refers to the sensor's id. The point `x` belongs to some of the allowed sensor positions on the car's surface. For each `x`, points `y` are sampled from the environment so that the distance between `x` and `y` is range `R_i` for sensor `i`.

We are dealing with different cases depending on how the car surface is positioned in the space. A bit simplifying we can say that one of the varibles, say `x`, runs over an interval. The values for the two other variables, `y` and `z`, we get from the plane equation when we substitute the value `x`. Now these triples belong to the car's surface. 

In [6]:
def sample_from_car_surface(corners, equation, fixed_intervals, car_sample_accuracy):
    sample = list()
    
    # This is the simple case that the plane is parallel with some of the three axis. 
    # Thus the parallel axis stays constant.
    # For example, side mirror, back, trunk and sides are parallel to one of the axis
    
    if len(fixed_intervals) == 2:
        if 'x' in fixed_intervals and 'y' in fixed_intervals:
            
            z = corners[0][2]
            
            for x in fixed_intervals['x']:
                for y in fixed_intervals['y']:
                    sample.append((x, y, z))
                    
        elif 'x' in fixed_intervals and 'z' in fixed_intervals:
            
            y = corners[0][1]
            
            for x in fixed_intervals['x']:
                for z in fixed_intervals['z']:
                    sample.append((x, y, z))
                    
        elif 'y' in fixed_intervals and 'z' in fixed_intervals:
            
            x = corners[0][0]
            
            for y in fixed_intervals['y']:
                for z in fixed_intervals['z']:
                    sample.append((x, y, z))

    elif len(fixed_intervals) == 1:
        
        if 'x' in fixed_intervals:
            y, z = symbols('y z')
            expr = equation['y']*y + equation['z']*z - equation['d']
            y_interval = None
            
            c1 = corners[0][1]
            c2 = corners[1][1]
            c3 = corners[2][1]
            
            if c1 < c2:
                y_interval = range(c1, c2, car_sample_accuracy)
            elif c1 > c2:
                y_interval = range(c2, c1, car_sample_accuracy)
            elif c1 < c3:
                y_interval = range(c1, c3, car_sample_accuracy)
            elif c1 > c3:
                y_interval = range(c3, c1, car_sample_accuracy)
        
            
            for x in fixed_intervals['x']:
                for y_var in y_interval:
                    y_expr = expr.subs(y, y_var)
                    z = math.floor(solve(y_expr)[0])
                    sample.append((x, y_var, z))
                    #print((x, y_var, z))
                x += car_sample_accuracy
                    
        elif 'y' in fixed_intervals:
            x, z = symbols('x z')
            expr = equation['x']*x + equation['z']*z - equation['d']
            x_interval = None
            
            c1 = corners[0][0]
            c2 = corners[1][0]
            c3 = corners[2][0]
            
            if c1 < c2:
                x_interval = range(c1, c2, car_sample_accuracy)
            elif c1 > c2:
                x_interval = range(c2, c1, car_sample_accuracy)
            elif c1 < c3:
                x_interval = range(c1, c3, car_sample_accuracy)
            elif c1 > c3:
                x_interval = range(c3, c1, car_sample_accuracy)
        
            
            for y in fixed_intervals['y']:
                for x_var in x_interval:
                    x_expr = expr.subs(x, x_var)
                    z = math.floor(solve(x_expr)[0])
                    sample.append((x_var, y, z))
                    #print((x_var, y, z))
                y += car_sample_accuracy
            
        elif 'z' in fixed_intervals:
                
            x, y = symbols('x y')
            expr = equation['x']*x + equation['y']*y - equation['d']
            x_interval = None

            c1 = corners[0][0]
            c2 = corners[1][0]
            c3 = corners[2][0]

            if c1 < c2:
                x_interval = range(c1, c2, car_sample_accuracy)
            elif c1 > c2:
                x_interval = range(c2, c1, car_sample_accuracy)
            elif c1 < c3:
                x_interval = range(c1, c3, car_sample_accuracy)
            elif c1 > c3:
                x_interval = range(c3, c1, car_sample_accuracy)


            for z in fixed_intervals['z']:
                for x_var in x_interval:
                    x_expr = expr.subs(x, x_var)
                    y = math.floor(solve(x_expr)[0])
                    #print(y)
                    sample.append((x_var, y, z))
                    #print((x_var, y, z))
                z += car_sample_accuracy
        
    return sample



After we have sampled points from the car's surface, we sample points from the environment. We fix a point `x` on the car's surface and then pick a point `y` from the environment so that the distance between `x` and `y` is the range `R_i`. We let the angle between the car's surface and the vector `x - y` run over different values which produces multiple possible point pairs `(x, y)`. In the optimal situation we could sample points with some sufficiently small accuracy factor.

In [7]:
for sensor in sensors:
    variables[sensor["id"]] = dict()
    for pos_name in allowed_sensor_positions:
        pos = allowed_sensor_positions[pos_name]
        #print(sensor_types)
        if sensor_types[sensor["type"]] in pos["allowed_sensors"]:
            
            variables[sensor["id"]][pos_name] = list()
            srange = sensor["view"]["range"]
            corners = pos["corners"]
            equation = pos["equation"]
            fixed_intervals = pos["fixed_intervals"]
            normal_vector = equation['normal_vector']
            spanning_vector_on_plane1 = equation['span1']
            spanning_vector_on_plane2 = equation['span2']
            
            car_sample = sample_from_car_surface(corners, equation, fixed_intervals, car_sample_accuracy)
            
            #print(pos_name)
            #print(car_sample)
            #scatter_plot(car_sample)
        
            for car_point in car_sample:
                for angle_on_plane in range(0, 360, angle_accuracy):
                    for angle_for_normal in range(0, 90, angle_accuracy):
                        
                        rad_angle_on_plane = math.radians(angle_on_plane)
                        rad_angle_for_normal = math.radians(angle_for_normal)
                        
                        #print("Car point ", car_point)
                        #print("Normal vector", normal_vector)
                        #print("Spanning vector", spanning_vector_on_plane1)
                        #print("Spanning vector", spanning_vector_on_plane2)
                        
                        # We start moving from the fixed point on the car's surface
                        point_in_environment = np.array([float(x) for x in car_point])
                        car_point_vector = np.array(car_point)
                        
                        #print(rad_angle_on_plane)
                        #print(math.cos(rad_angle_on_plane))
                        
                        #print("#11 ", srange*math.cos(rad_angle_on_plane)*spanning_vector_on_plane1)
                        #print("#12 ", srange*math.sin(rad_angle_on_plane)*spanning_vector_on_plane2)
                        #print("#13 ", srange*math.sin(rad_angle_for_normal)*normal_vector)
                        
                        # Move along the plane to the direction of the first spanning vector
                        point_in_environment += srange*math.cos(rad_angle_on_plane)*math.cos(rad_angle_for_normal)*spanning_vector_on_plane1
                        
                        #print("#1 ", point_in_environment)
                        
                        # Move along the plane to the direction of the second spanning vector
                        point_in_environment += srange*math.sin(rad_angle_on_plane)*math.cos(rad_angle_for_normal)*spanning_vector_on_plane2
                        
                        #print("#2 ", point_in_environment)
                        
                        # Move to the orthonogal direction to the plane i.e. "upwards"
                        point_in_environment += srange*math.sin(rad_angle_for_normal)*normal_vector
                        
                        #print("#3 ", point_in_environment)
                        
                        #env_points.append(point_in_environment)
                        #scatter_plot(env_points)
                        
                        point_in_environment = [math.floor(x) for x in point_in_environment]
                        
                        # For bugging purposes:
                        #print("Angles are ", angle_on_plane, angle_for_normal)
                        #print("Distance defined in the sensor data is " + str(srange) + " and the distance between the sampled points is ", euclidean_3d_distance(car_point, tuple(point_in_environment)))
            
                        b_variable = (car_point, tuple(point_in_environment), sensor["id"])
                        variables[sensor["id"]][pos_name].append(b_variable)
                        all_variables.append(b_variable)
            
print("Number of variables: ",len(all_variables))


Number of variables:  2412


## Constructing quadratic unconstrained binary optimization model

In [8]:
vartype = dimod.BINARY
main_bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, vartype)

The following functions enable us to append or initialize coefficients for the variables in the BQM. The distance function implements the ordinary Euclidean distance.

In [9]:
def append_linear_safe(variable, value, linear_dict):
    if variable in linear_dict.keys():
        linear_dict[variable] = linear_dict[variable] + value
    else:
        linear_dict[variable] = value

def append_quadratic_safe(variable, value, quadratic_dict):
    if variable in quadratic_dict.keys():
        quadratic_dict[variable] = quadratic_dict[variable] + value
    else:
        quadratic_dict[variable] = value

def sensor_view_volume(sensor_id):
    for sensor in sensors:
        if sensor['id'] == sensor_id:
            # Following code calculates volume of view of sensor
            side1 = 2*sensor["view"]["range"]*float(math.tan(math.radians(sensor["view"]["angle"]["horizontal"])/2))
            side2 = 2*sensor["view"]["range"]*float(math.tan(math.radians(sensor["view"]["angle"]["vertical"])/2))
            return (1/3)*sensor["view"]["range"]*side1*side2
        
def print_current_qubo(number_of_linear_terms = 100, number_of_quadratic_terms = 100):
    i = 0
    for elem in main_bqm.linear:
        print(elem, main_bqm.linear[elem])
        i+=1
        if i > number_of_linear_terms:
            break

    i = 0
    for elem in main_bqm.quadratic:
        print(elem, main_bqm.quadratic[elem])
        i+=1
        if i > number_of_quadratic_terms:
            break

### Constraint 1: selecting sufficiently sensors to cover the environment

Every binary quadratic function which is part of the model contains four parameters: linear terms, quadratic terms, offset (constant) and variable type. Variable type is always BINARY since we are using QUBO. If we use Ising, we set variable type to be SPIN.

In [10]:
# Encoding constraint H1

E = environment_x*environment_y*environment_z
print("Total volume of the environment: ", E)

A1 = 1/math.pow(10, 12)
linear_h1 = {}
quadratic_h1 = {}
offset_h1 = float(math.pow(E, 2))
#print(offset_h1)


# Linear terms
for sensor_id in variables:
    
    volume = sensor_view_volume(sensor_id)
    print("Volume of sensor " + str(sensor_id) + ": ", volume)
    coefficient = float(math.pow(volume, 2) - 2*E*volume)
    
    for surface in variables[sensor_id]:
        for var in variables[sensor_id][surface]:
            append_linear_safe(var, coefficient, linear_h1)


# Quadratic terms
quadratic_terms = combinations(all_variables, 2)

for pair in quadratic_terms:
    b1 = pair[0]
    b2 = pair[1]
    
    volume1 = sensor_view_volume(b1[2])
    volume2 = sensor_view_volume(b2[2])
    
    coefficient_quadratic = float(2*volume1*volume2)
    append_quadratic_safe((b1, b2), coefficient_quadratic, quadratic_h1)
    
            
bqm_h1 = dimod.BinaryQuadraticModel(linear_h1, quadratic_h1, offset_h1, vartype)
bqm_h1.scale(A1)
main_bqm.update(bqm_h1)

Total volume of the environment:  136374000000
Volume of sensor 1:  435164093.9080988
Volume of sensor 2:  6799438967.314044
Volume of sensor 3:  18657660526.309734
Volume of sensor 4:  54395511738.51235
Volume of sensor 5:  54395511738.51235


The following code is for printing the linear and quadratic terms.

In [11]:
print_current_qubo(10, 10)

((-2200, -550, 1745), (-1874, -2509, 1508), 1) -118500768.49661927
((-2200, -550, 1745), (-3054, -1530, 224), 1) -118500768.49661927
((-2200, -550, 1745), (-2314, -1600, 1827), 1) -118500768.49661927
((-2200, -550, 1745), (-3274, -1075, 383), 1) -118500768.49661927
((-2200, -550, 1745), (-2641, 359, 2064), 1) -118500768.49661927
((-2200, -550, 1745), (-3437, -96, 502), 1) -118500768.49661927
((-2200, -550, 1745), (-2527, 1408, 1981), 1) -118500768.49661927
((-2200, -550, 1745), (-3380, 429, 461), 1) -118500768.49661927
((-2200, -550, 1745), (-2087, 499, 1662), 1) -118500768.49661927
((-2200, -550, 1745), (-3160, -26, 301), 1) -118500768.49661927
((-2200, -550, 1745), (-1760, -1460, 1425), 1) -118500768.49661927
((-2200, -550, 1745), (-2997, -1005, 183), 1) -118500768.49661927
((-2200, -50, 1745), (-1874, -2009, 1508), 1) -118500768.49661927
((-2200, -50, 1745), (-3054, -1030, 224), 1) -118500768.49661927
((-2200, -50, 1745), (-2314, -1100, 1827), 1) -118500768.49661927
((-2200, -50, 17

### Constraint 2: optimizing overlap of sensor views

One of the biggest problems in the current code is the following overlap function. It is just a rough estimate on how much two sensor views overlap.

In [13]:
def sensor_inside_sensor(car_p1, env_p1, car_p2, env_p2):
    if car_p2.all() == env_p2.all():
        return True
    if car_p1.all() == car_p2.all():
        if (car_p2 - env_p2).all() != 0:
            line = (car_p1 - env_p1)/(car_p2 - env_p2)
            for i in range(len(line) - 1):
                if line[i] != line[i+1]:
                    return False
            else:
                return True
    return False

def overlap(b1, b2):
    car_p1 = np.array([int(x) for x in b1[0]])
    env_p1 = np.array([int(x) for x in b1[1]])
    id1 = int(b1[2])
    
    car_p2 = np.array([int(x) for x in b2[0]])
    env_p2 = np.array([int(x) for x in b2[1]])
    id2 = int(b2[2])
    
    if sensor_inside_sensor(car_p1, env_p1, car_p2, env_p2):
        return 1
    
    cone_axis1 = env_p1 - car_p1
    cone_axis2 = env_p2 - car_p2
    
    # Angle between the axis
    
    u_cone_axis1 = cone_axis1 / np.linalg.norm(cone_axis1)
    u_cone_axis2 = cone_axis2 / np.linalg.norm(cone_axis2)
    
    axis_angle = np.arccos(np.dot(u_cone_axis1, u_cone_axis2))
    
    #print(math.degrees(axis_angle))
    
    # Distance between the middle points of the vectors cone_axis1 and cone_axis2
    
    mid_cone_axis1 = (np.linalg.norm(cone_axis1)/2)*u_cone_axis1
    mid_cone_axis2 = (np.linalg.norm(cone_axis2)/2)*u_cone_axis2
    
    mid_point_dist = np.linalg.norm(mid_cone_axis1 - mid_cone_axis2)
    
    top_point_dist = np.linalg.norm(env_p1 - env_p2)
    
    #print(mid_point_dist)
    
    sensor1, sensor2 = None, None
    
    for sensor in sensors:
        
        if sensor['id'] == id1:
            sensor1 = sensor
        
        if sensor['id'] == id2:
            sensor2 = sensor
            
    # This part of the code is very heuristical and possibly works badly and ruins everything
    
    side1 = np.linalg.norm(cone_axis1)
    side2 = np.linalg.norm(cone_axis2)
    
    top_half1_dist1 = math.tan(math.radians(sensor1["view"]["angle"]["horizontal"])/2)*side1
    top_half1_dist2 = math.tan(math.radians(sensor1["view"]["angle"]["vertical"])/2)*side1
    
    top_half2_dist1 = math.tan(math.radians(sensor2["view"]["angle"]["horizontal"])/2)*side2
    top_half2_dist2 = math.tan(math.radians(sensor2["view"]["angle"]["vertical"])/2)*side2
    
    #print("Distances on top: ", top_half1_dist1, top_half1_dist2, top_half2_dist1, top_half2_dist2)
    
    mid_half1_dist1 = top_half1_dist1/2
    mid_half1_dist2 = top_half1_dist2/2
    
    mid_half2_dist1 = top_half2_dist1/2
    mid_half2_dist2 = top_half2_dist2/2
    
    #print("Distances in middle: ", mid_half1_dist1, mid_half1_dist2, mid_half2_dist1, mid_half2_dist2)
    
    difference_top1 = top_half1_dist1 + top_half2_dist1 - top_point_dist
    difference_top2 = top_half1_dist2 + top_half2_dist2 - top_point_dist
    
    difference_mid1 = mid_half1_dist1 + mid_half2_dist1 - mid_point_dist
    difference_mid2 = mid_half1_dist2 + mid_half2_dist2 - mid_point_dist
    
    #print("Top differences: ", difference_top1, difference_top2)
    #print("Middle differences: ", difference_mid1, difference_mid2)
    
    top_divisor1 = max([top_half1_dist1, top_half2_dist1])
    top_divisor2 = max([top_half1_dist2, top_half2_dist2])
    
    top_divisor = top_divisor1 + top_divisor2
    
    mid_divisor1 = max([mid_half1_dist1, mid_half2_dist1])
    mid_divisor2 = max([mid_half1_dist2, mid_half2_dist2])
    
    mid_divisor = mid_divisor1 + mid_divisor2
    
    top_sum = 0
    mid_sum = 0
    
    if difference_top1 > 0:
        top_sum += difference_top1
    if difference_top2 > 0:
        top_sum += difference_top2
    if difference_mid1 > 0:
        mid_sum += difference_mid1
    if difference_mid2 > 0:
        mid_sum += difference_mid2
    
    overlap_result = (top_sum + mid_sum)/(top_divisor + mid_divisor)
    
    #print("Final result: ", overlap_result)
    
    if overlap_result > 1:
        return 1/overlap_result
    
    return overlap_result

In [14]:
# Encoding constraint H2

A2 = 1000
linear_h2 = {}
quadratic_h2 = {}
offset_h2 = 0

quadratic_terms = combinations(all_variables, 2)

for pair in quadratic_terms:
    
    b1 = pair[0]
    b2 = pair[1]
    
    coefficient_quadratic = overlap(b1, b2)
    
    append_quadratic_safe((b1, b2), coefficient_quadratic, quadratic_h2)
            
bqm_h2 = dimod.BinaryQuadraticModel(linear_h2, quadratic_h2, offset_h2, vartype)
bqm_h2.scale(A2)
main_bqm.update(bqm_h2)
    

<ipython-input-13-1b03c232f8a0>:34: RuntimeWarning: invalid value encountered in arccos
  axis_angle = np.arccos(np.dot(u_cone_axis1, u_cone_axis2))


In [15]:
print_current_qubo(50, 50)

((-2200, -550, 1745), (-1874, -2509, 1508), 1) -118500768.49661927
((-2200, -550, 1745), (-3054, -1530, 224), 1) -118500768.49661927
((-2200, -550, 1745), (-2314, -1600, 1827), 1) -118500768.49661927
((-2200, -550, 1745), (-3274, -1075, 383), 1) -118500768.49661927
((-2200, -550, 1745), (-2641, 359, 2064), 1) -118500768.49661927
((-2200, -550, 1745), (-3437, -96, 502), 1) -118500768.49661927
((-2200, -550, 1745), (-2527, 1408, 1981), 1) -118500768.49661927
((-2200, -550, 1745), (-3380, 429, 461), 1) -118500768.49661927
((-2200, -550, 1745), (-2087, 499, 1662), 1) -118500768.49661927
((-2200, -550, 1745), (-3160, -26, 301), 1) -118500768.49661927
((-2200, -550, 1745), (-1760, -1460, 1425), 1) -118500768.49661927
((-2200, -550, 1745), (-2997, -1005, 183), 1) -118500768.49661927
((-2200, -50, 1745), (-1874, -2009, 1508), 1) -118500768.49661927
((-2200, -50, 1745), (-3054, -1030, 224), 1) -118500768.49661927
((-2200, -50, 1745), (-2314, -1100, 1827), 1) -118500768.49661927
((-2200, -50, 17

### Constraint 3: minimizing total price

In [16]:
# Encoding constraint H3

A3 = 1
linear_h3 = {}
quadratic_h3 = {}
offset_h3 = 0

for variable in all_variables:
    sensor_id = variable[2]
    price = get_sensor_price(sensor_id)
    append_linear_safe(variable, price, linear_h3)

            
bqm_h3 = dimod.BinaryQuadraticModel(linear_h3, quadratic_h3, offset_h3, vartype)
bqm_h3.scale(A3)
main_bqm.update(bqm_h3)

In [17]:
print_current_qubo(50, 50)

((-2200, -550, 1745), (-1874, -2509, 1508), 1) -118500668.49661927
((-2200, -550, 1745), (-3054, -1530, 224), 1) -118500668.49661927
((-2200, -550, 1745), (-2314, -1600, 1827), 1) -118500668.49661927
((-2200, -550, 1745), (-3274, -1075, 383), 1) -118500668.49661927
((-2200, -550, 1745), (-2641, 359, 2064), 1) -118500668.49661927
((-2200, -550, 1745), (-3437, -96, 502), 1) -118500668.49661927
((-2200, -550, 1745), (-2527, 1408, 1981), 1) -118500668.49661927
((-2200, -550, 1745), (-3380, 429, 461), 1) -118500668.49661927
((-2200, -550, 1745), (-2087, 499, 1662), 1) -118500668.49661927
((-2200, -550, 1745), (-3160, -26, 301), 1) -118500668.49661927
((-2200, -550, 1745), (-1760, -1460, 1425), 1) -118500668.49661927
((-2200, -550, 1745), (-2997, -1005, 183), 1) -118500668.49661927
((-2200, -50, 1745), (-1874, -2009, 1508), 1) -118500668.49661927
((-2200, -50, 1745), (-3054, -1030, 224), 1) -118500668.49661927
((-2200, -50, 1745), (-2314, -1100, 1827), 1) -118500668.49661927
((-2200, -50, 17

### Solve QUBO

Unfortunately, LeapHybridSampler is not available in Amazon Bracket. That is why this code will not work in Bracket. On the other hand, we tried run the code using BracketDwaveSampler but the problem cannot be mapped to the circuits.

In [ ]:
main_bqm.normalize()

sampler = LeapHybridSampler()
sampleset = sampler.sample(main_bqm)
sample = sampleset.first.sample
print(sampleset)
print()
# energy = sampleset.first.energy
print("Possible sensor positions in the space (point on car, point in environment, sensor id):")
i = 0
for varname, value in sample.items():
    if value == 1:
        i+=1
        print(varname, value)
print(i)